# **ENOE $^N$: Un Análisis Geoespacial de la Informalidad en México (2022)**

---

## **Metodología para la medición de Informalidad**

Con base en *Conociendo la base de datos de la ENOE. Datos ajustados a proyecciones de población 2010. (2020)*, INEGI.


> **Resumen de la base de datos de la ENOE**: Por cada vivienda existe uno o más registros de hogares, para cada hogar con entrevista completa (más
adelante se defi ne entrevista completa) se cuenta con el registro de las características sociodemográfi cas
de cada residente, es decir, cada hogar tiene a sus residentes en la tabla de sociodemográfico (SDEM)
y para cada residente con edad de 12 años cumplidos ó más se cuenta con un registro del Cuestionario
de Ocupación y Empleo, el cual se almacena en las tablas COE1 (de batería 1 a 5) y COE2 (de batería
6 a la última).

Basándose en lo anterior, se concluye que es posible identificar a los **residentes de un hogar** en la tabla de **caracteristicas sociodemográficas** (`SDEM`) dentro de la base de datos de la **ENOE**. Es importante señalar que cada **fila/registro** dentro de esta última, corresponde a las cáracteristicas de una única persona. Sin embargo, se puede conocer la **Frecuencia absoluta expandida** i.e. el número de personas que cuentan con las mismas mediante el **factor de expansión** en el **campo/columna** `FAC`.

> “El factor de expansión es el coeficiente que le otorga determinado peso en la muestra en función de su representatividad de otros tantos casos 
similares a él tomando en cuenta su estrato socioeconómico y lugar de residencia”.

$$ registro_i \times FAC = \text{Frecuencia absoluta expandida} \tag{1}$$

**Nota**: En la nueva base de datos **ENOE $^N$**, la variable `FAC` es reemplazada por `fac_tri` y `fac_mes`

En términos técnicos, es posible identificar a la población en el **sector informal** al filtrar por los registros que, en el campo de **Tipo de Unidad Económica 2** (`TUE2`) de la tabla `SDEM` cuenten con un valor de **5**.

```SQL

* Con esta sentencia se obtiene a la población económicamente activa.
SUM FAC FOR R_DEF=’00’ AND (C_RES=1 OR C_RES=3) AND (EDA>=’15’ AND EDA<=’98’) AND
CLASE1=1

* Para calcular los indicadores estratégicos e.g. Población en informalidad, se les aplica
* el pre-criterio siguiente:
R_DEF=’00’ AND (C_RES=’1’ OR C_RES=’3’) AND (EDA>=’15’ AND EDA<=’98’)

* Con esta se identificaría a la población en el sector informal
SELECT sum FAC FROM SDEM ... AND TUE2 = 5
```

Dando continuidad a la parte técnica, algo que se destaca es que, además del filtro de `TUE=5`, el cuál nos permite identificar personas dentro del **sector informal**, otras variables y criterios son empleados para encontrar a estos residentes: `R_DEF`, `C_RES` y `EDA`. El **INEGI** los describe como:

- > `R_DEF`: Existe un campo en la tabla HOGARES con el nombre “R_DEF” que almacena el resultado definitivo
de la entrevista del hogar, una ENTREVISTA COMPLETA se determina cuando en el campo “R_DEF”
almacena el valor ‘00’ (doble cero). Todo valor diferente a 00, es considerado como NO ENTREVISTA (ver documento de captación CARACTERISTICAS SOCIODEMOGRAFICAS en el apartado resultados de la entrevista).

- > `C_RES`:  En la tabla SOCIODEMOGRÁFICO existe el campo “C_RES”, que determina la condición de residencia
del ocupante del hogar. Si el valor que almacena este campo es 1, se trata de un residente habitual; si el
valor es 2, se trata de ausente definitivo; si el valor es 3, es un nuevo residente. Para mayor información
ver documento metodológico de la ENOE.

- > `EDA`:  En el campo “EDA”, se almacenan los años cumplidos. Si la edad del residente es de 12 años y más y
además tiene un 1 o un 3 en el campo “C_RES”, debe de existir un registro en la tabla CUESTIONARIO
DE OCUPACIÓN Y EMPLEO, si no cumple con estas características se termina su secuencia en la tabla
SOCIODEMOGRÁFICO.

Con base en lo anterior, se concluye que para que **un residente** sea considerado para calcular **indicadores estratégicos** es necesario que este: **1)** cuente con **entrevista completa** (`R_DEF = '00'`), **2)** sea **residente habitual** o **nuevo** (`C_RES` = '1' OR `C_RES` = '3'), y **3)** su edad se encuentre dentro de los **15** y **98** años (`EDA` >= '15' AND `EDA` <= '98').

Para calcular la **tasa de desocupación en el sector informal**, utilizado como un **acercamiento y comparativo** de la **informalidad** a nivel **entidad federativa** y **municipio**, se utiliza la formula definida por la **INEGI** dentro de su documento *Conociendo la base de datos de la ENOE (2020)*, (p.27):

**VIII.-Tasa de ocupación en el sector informal**

```SQL
Universo: R_DEF=00 y C_RES=1 ó 3 y edad=15-98
```

$$ TOSI = \frac{POSI}{PO} \times 100 \tag{2}$$

**Donde**:
- **POSI**: Población ocupada en el sector informal (`TUE2=5`)
- **PO**: Población ocupada (`Clase2 = 1`)

## **Fuentes**

- INEGI. (2020). *Cómo se hace la ENOE. Métodos y Procedimientos*. https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/nueva_estruc/702825190613.pdf
- INEGI. (2020). *Conociendo la base de datos de la ENOE. Datos ajustados a proyecciones de población 2010.*. https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/doc/con_basedatos_proy2010.pdf
- INEGI. (2021). *Encuesta Nacional de Ocupación y Empleo Nueva Edición (ENOE $^N$) Estructura de la base de datos*. https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/doc/enoe_n_321_fd_c_bas_amp.pdf 
- [TBC](https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/metodologias/ENOE/ENOE2014/informal_laboral/702825060459.pdf)

### **Bases de datos**
- Conjunto de datos + metadatos: https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/datosabiertos/2022/conjunto_de_datos_enoen_2022_1t_csv.zip
- Conjunto de datos simplificado (solo tablas en excel): https://www.inegi.org.mx/programas/enoe/15ymas/#Microdatos

### **Benchmark del 1er trimestre**
- Encuesta Nacional de Ocupación y Empleo (Nueva Edición) ENOE N. Resultados del primer trimestre de 2022. https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/doc/enoe_n_presentacion_ejecutiva_trim1_2022.pdf
- Comunicado de Prensa. Encuesta Nacional de Ocupación y Empleo, Nueva Edición. Primer Trimestre de 2022. https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/doc/enoe_n_nota_tecnica_trim1_2022.pdf

# **TO DO**:

- Elaborar mapas para ubicar la informalidad en México
- Ubicar las localidades/municipios con mayor porcentaje de población en el sector informal
- Identificar las zonas rurales y urbanas con mayor porcentaje de población en el sector informal
- Analizar características socioeconómicas relevantes por región y presentarlas en los mapas

# **NICE2HAVE**
- Automatizar la descargar y dirs de `data`


In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt